# NextHack

https://ds.sbidea.ru/data

Участникам соревнования необходимо предсказать вероятность увольнения менеджера по продажам в следующие три месяца.

Для построения модели предлагается использовать следующие данные:
- SAP о структурной позиции сотрудника, наличии выговоров, отпусков, больничных, производительности и т. п. 
- Единый Профиль Клиента по сотруднику, как Клиенту Банка (активность использования продуктов Банка),
- информацию о входах-выходах сотрудников во внутреннюю сеть Банка и суммарном нахождении в сети за день.

Оценка алгоритма - ROC AUC

In [1]:
import pandas as pd
import numpy as np

In [36]:
train = pd.read_csv("train_small.csv")
test = pd.read_csv("test.csv")

C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (0,2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Анализ данных, заполнение пропусков

In [37]:
train.head()

,C_Year,EtalonPosName,R_Year,age_qty,boss_pos_name,boss_subord_1y_max_qty,boss_subord_1y_min_qty,boss_subord_in_1m_qty,boss_subord_in_1y_qty,boss_subord_out_1m_qty,...,up_day_nv_sum_3m,up_day_nv_sum_6m,vl_nv_KBP_sum_3m,vl_nv_KBP_sum_6m,vl_nv_MP_sum_3m,vl_nv_MP_sum_6m,vl_nv_sum_3m,vl_nv_sum_6m,vsp_cd,work_rate_pct
0,C,0.0,D,26.0,0.0,33.0,31.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17,1.0
1,C,0.0,C,25.0,0.0,33.0,28.0,1.0,14.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1189,1.0
2,C,0.0,C,44.0,0.0,17.0,13.0,2.0,15.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7,1.0
3,B,0.0,C,24.0,0.0,21.0,1.0,0.0,33.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,543,1.0
4,C,0.0,C,25.0,0.0,15.0,11.0,1.0,7.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,371,1.0


In [38]:
train.isnull().sum()

C_Year                            27672
EtalonPosName                      1110
R_Year                            27708
age_qty                              14
boss_pos_name                      1272
boss_subord_1y_max_qty             1460
boss_subord_1y_min_qty             1460
boss_subord_in_1m_qty              1460
boss_subord_in_1y_qty              1460
boss_subord_out_1m_qty             1460
boss_subord_out_1y_qty             1460
boss_subord_qty                    1460
boss_tabnum                           0
crd_cc_act_qty                      238
crd_cc_act_spend_qty                238
crd_cc_ever_nflag                   127
crd_cc_mnth_snc_issue_frac        34194
crd_cc_mnth_snc_issue_qty         34193
crd_cc_overdue_nflag                127
crd_cc_own_bal_rub_amt            17776
crd_cc_pos_auto_qty               87920
crd_cc_pos_auto_rub_amt           87920
crd_cc_pos_beuaty_qty             92464
crd_cc_pos_beuaty_rub_amt         92464
crd_cc_pos_clothes_qty            84938


В обучающих и тестовых данных много пропусков, нужно их заполнить. Вычисляем медианы по столбцам обучающей выборки:

In [39]:
medians = train.median()

In [40]:
train.fillna(medians, inplace=True)
test.fillna(medians, inplace=True);

Смотрим, какие столбцы остались с пропусками:

In [41]:
train.columns[train.isnull().sum() > 0]

Index(['C_Year', 'R_Year', 'crd_cc_pos_othr_qty', 'crd_cc_pos_othr_rub_amt',
       'crd_dc_pos_othr_qty', 'crd_dc_pos_othr_rub_amt', 'gender',
       'lbt_payroll_min_dt', 'srv_sbol_atm_1st_login_ever_dt',
       'srv_sbol_mob_1st_login_ever_dt', 'srv_sbol_web_1st_login_ever_dt'],
      dtype='object')

Удаляем столбцы с датами и числовые столбцы, значения которых не удалось заполнить:

In [43]:
cols_to_delete = ['crd_cc_pos_othr_qty', 'crd_cc_pos_othr_rub_amt', 'crd_dc_pos_othr_qty', 'crd_dc_pos_othr_rub_amt', 
                  'lbt_payroll_min_dt', 'srv_sbol_atm_1st_login_ever_dt','srv_sbol_mob_1st_login_ever_dt', 
                  'srv_sbol_web_1st_login_ever_dt', 'report_dt']

In [44]:
train.drop(cols_to_delete, axis=1, inplace=True)
test.drop(cols_to_delete, axis=1, inplace=True)

Категориальные фичи кодируем с помощью one-hot encoding:

In [45]:
train = pd.get_dummies(train)
test = pd.get_dummies(test)

Выясняем, что выборка перекошенная &mdash; уволилось на два порядка меньше людей, чем осталось:

In [46]:
train["target_churn_lag1m"].value_counts()

0.0    96402
1.0     3598
Name: target_churn_lag1m, dtype: int64

### Подготовка выборки для валидации

Для валидации будем использовать те записи из тренировочной выборки, которые наиболее похожи на записи из тестовой выборки.

In [47]:
train["test"] = 0
test["test"] = 1

In [48]:
data = train.append(test).reset_index(drop=True)

In [49]:
X = data.drop(["target_churn_lag1m", "test"], axis=1)
y = data["test"]

In [50]:
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_predict

In [51]:
clf = XGBClassifier()

In [ ]:
%%time

pred = cross_val_predict(clf, X, y, cv=5, verbose=2, method='predict_proba')

In [27]:
th = 0.999

In [ ]:
val_index = X[(pred[:, 1] > th) & (data["test"] == 0)].index

In [ ]:
# train.loc[val_index, :].drop("test", axis=1).to_csv("_validation.csv", index=False)

In [ ]:
# train[~train.index.isin(val_index)].drop("test", axis=1).to_csv("_train2.csv", index=False)

In [55]:
train = pd.read_csv('_train2.csv')
test = pd.read_csv('_validation.csv')

C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (0,2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Создание новых фич

In [56]:
wtd = pd.read_csv("work_time_day.csv", index_col='tabnum')
wd = pd.read_csv("work_dur.csv", index_col='tabnum')

In [57]:
def prepare_data(data, has_churn=True):
    data = data.merge(wtd, left_on='tabnum', right_index=True, how='left')
    data.drop(['work_time_day_max', 'work_time_day_min'], axis=1, inplace=True)

    data = data.merge(wd, left_on='tabnum', right_index=True, how='left')
    data.drop(['work_dur_max', 'work_dur_min'], axis=1, inplace=True)

    data.drop(['tabnum'], axis=1, inplace=True)

    data["f3"] = data['crd_inf_cash_adv_rub_amt'] / data["lbt_inf_payroll_rub_amt"]
    data["f4"] = data['crd_inf_ext_transf_rub_amt_sum_6m'] / data["lbt_inf_payroll_rub_amt"]
    data["f5"] = data['crd_inf_pos_ret_rub_amt'] / data["lbt_inf_payroll_rub_amt"]
    data["f6"] = data['crd_otf_cc_pos_spend_rub_amt'] / data["lbt_inf_payroll_rub_amt"]
    data["f7"] = data['crd_otf_cc_total_rub_amt'] / data["lbt_inf_payroll_rub_amt"]
    data["f8"] = data['crd_otf_dc_cash_rub_amt'] / data["lbt_inf_payroll_rub_amt"]
    data["f9"] = data['crd_otf_dc_pos_spend_rub_amt'] / data["lbt_inf_payroll_rub_amt"]
    data["f10"] = data['crd_otf_pos_spend_rub_amt'] / data["lbt_inf_payroll_rub_amt"]
    data["f11"] = data['crd_otf_total_rub_amt_sum_6m'] / data["lbt_inf_payroll_rub_amt"]
    data["f12"] = data['crd_otf_trnf_pers_rub_amt'] / data["lbt_inf_payroll_rub_amt"]
    data["f13"] = data['crd_otf_trnf_pers_sbol_rub_amt'] / data["lbt_inf_payroll_rub_amt"]
    data["f14"] = data['crd_txn_spnd_3m_amt'] / data["lbt_inf_payroll_rub_amt"]
    data["f15"] = data['cred_mg_act_initial_rub_amt'] / data["lbt_inf_payroll_rub_amt"]
    data["f16"] = data['cred_mg_os_rub_amt'] / data["lbt_inf_payroll_rub_amt"]
    data["f17"] = data['cred_tot_act_initial_rub_amt'] / data["lbt_inf_payroll_rub_amt"]
    data["f18"] = data['cred_tot_mnth_pmt_rub_amt'] / data["lbt_inf_payroll_rub_amt"]
    data["f19"] = data['cred_tot_os_rub_amt'] / data["lbt_inf_payroll_rub_amt"]
    data["f20"] = data['ext_client_app_income_amt'] / data["lbt_inf_payroll_rub_amt"]
    data["f21"] = data['lbt_acct_dc_bal_rub_amt'] / data["lbt_inf_payroll_rub_amt"]
    data["f22"] = data['lbt_acct_dep_ca_bal_rub_amt'] / data["lbt_inf_payroll_rub_amt"]
    data["f23"] = data['lbt_acct_dep_tot_bal_rub_amt'] / data["lbt_inf_payroll_rub_amt"]
    data["f24"] = data['lbt_acct_tot_bal_rub_amt'] / data["lbt_inf_payroll_rub_amt"]
    data["f25"] = data['lbt_acct_tot_davg_mnth_rub_amt'] / data["lbt_inf_payroll_rub_amt"]
    data["f26"] = data['lbt_inf_intr_cap_rub_amt'] / data["lbt_inf_payroll_rub_amt"]
    data["f27"] = data['lbt_inf_topup_rub_amt'] / data["lbt_inf_payroll_rub_amt"]
    data["f28"] = data['lbt_inf_total_rub_amt'] / data["lbt_inf_payroll_rub_amt"]
    data["f29"] = data['lbt_payroll_l6m_tot_rub_amt'] / data["lbt_inf_payroll_rub_amt"]
    data["f30"] = data['lbt_payroll_tax_payer_cd'] / data["lbt_inf_payroll_rub_amt"]
    data["f31"] = data['lne_mg_debt_os_rub_amt'] / data["lbt_inf_payroll_rub_amt"]
    data["f32"] = data['lne_pl_debt_os_rub_amt'] / data["lbt_inf_payroll_rub_amt"]
    data["f33"] = data['lne_tot_max_ovrd_ever_rub_amt'] / data["lbt_inf_payroll_rub_amt"]
    data["f34"] = data['srv_ap_txn_amt'] / data["lbt_inf_payroll_rub_amt"]
    data["f35"] = data['srv_sbol_txn_3m_amt'] / data["lbt_inf_payroll_rub_amt"]
    data["f36"] = data['tot_sld_up_avg_6m'] / data["lbt_inf_payroll_rub_amt"]
    data["f37"] = data['tot_sld_up_max6m'] / data["lbt_inf_payroll_rub_amt"]
    data["f38"] = data['tot_sld_up_min6m'] / data["lbt_inf_payroll_rub_amt"]
    data["f39"] = data['tot_sld_up_sum_6m'] / data["lbt_inf_payroll_rub_amt"]
    data["f40"] = data['total_sold_up'] / data["lbt_inf_payroll_rub_amt"]

    if has_churn:
        return data.drop(['target_churn_lag1m'], axis=1), data['target_churn_lag1m']
    else:
        return data

In [58]:
X_train, y_train = prepare_data(train)
X_test, y_test = prepare_data(test)

### Отбор признаков

Пробовались разные варианты:

- алгоритм Boruta

In [ ]:
from boruta import BorutaPy
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf = RandomForestClassifier(max_depth=5)

In [ ]:
feat_selector = BorutaPy(rf, n_estimators='auto', random_state=1)

In [ ]:
%%time

feat_selector.fit(X_train, y_train)

In [ ]:
col_names = X_train.columns[feat_selector.support_]

- Recursive feature elimination

In [61]:
from sklearn.feature_selection import RFE

In [ ]:
rfe = RFE(XGBClassifier(), n_features_to_select=1, verbose=5)

In [ ]:
%%time

rfe.fit(X_train, y_train)

In [ ]:
ranking = rfe.ranking_

In [ ]:
X_train.columns[ranking.argsort()]

- жадный поиск

In [ ]:
%%time

max_score = 0
col_mask = [False] * len(X_train.columns)
ranking_argsort = ranking.argsort()

for i in range(len(ranking_argsort)):
    col_mask[ranking_argsort[i]] = True
    clf = XGBClassifier()
    clf.fit(X_train[X_train.columns[col_mask]], y_train)
    score = roc_auc_score(y_test, clf.predict_proba(X_test[X_test.columns[col_mask]])[:, 1])
    print(i+1, score, "" if score > max_score else "skipping")
    if score > max_score:
        max_score = score
    else:
        col_mask[ranking_argsort[i]] = False

In [ ]:
col_names = X_test.columns[col_mask]

### Борьба с перекошенной обучающей выборкой

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

In [ ]:
%%time

prediction = np.zeros(y_test.shape[0])
n_estimators = 500

for i in range(n_estimators):
    if i % 100 == 0:
        print("predictions " + str(i + 1) + "-" + str(min(i + 100, n_estimators)))
    rus = RandomUnderSampler()
    X_resampled, y_resampled = rus.fit_resample(X_train, y_train)
    clf = XGBClassifier()
    clf.fit(X_resampled, y_resampled)
    prediction += clf.predict_proba(X_test.values)[:,1]
    
prediction /= n_estimators

### Подбор гиперпараметров

In [ ]:
%%time

clf = XGBClassifier(n_estimators=10000, reg_lambda=70, gamma=4)

clf.fit(X_train, y_train, early_stopping_rounds=30, eval_metric="auc", \
        eval_set=[(X_train, y_train), (X_test, y_test)], verbose=True)

In [ ]:
roc_auc_score(y_test, prediction)

Было проведено множество экспериментов, составлялась таблица:

<img src="2019-09-09_192435.png"/>

Отправки в проверяющую систему кратко описывались:

<img src="2019-09-09_192751.png"/>

### Подготовка решения к отправке

In [ ]:
train = pd.read_csv('train.csv')

In [ ]:
X_train, y_train = prepare_data(train)

In [ ]:
%%time

clf = XGBClassifier(n_estimators=240, reg_alpha=15)

clf.fit(X_train, y_train)

In [ ]:
test = pd.read_csv("test.csv")

In [ ]:
X_test = prepare_data(test, False)

In [ ]:
prediction = clf.predict_proba(X_test)[:,1]

In [ ]:
pd.DataFrame({'tabnum': test['tabnum'], 'prob': prediction}).to_csv("submission.csv")